목표 : 상장기업 결측치 채우기
=====================
- inf는 max로 대체, -inf는 min으로 대체
- 정규성 검사
- 정규성 -> fillna(mean 평균)
- 정규성X -> fillna(median 중앙값)

import modules

In [1]:
import pandas as pd
import numpy as np

#워닝 메시지
import warnings
warnings.filterwarnings('ignore')

import os
current_directory = os.getcwd()
print("현재 디렉토리:", current_directory)

from scipy.stats import shapiro, anderson, kstest, norm

현재 디렉토리: /Users/haph/Study/ubion/Project2/상장/2.데이터전처리


환경설정

In [58]:
# pd.set_option('display.max_rows', None)
pd.reset_option('display.max_rows')

# pd.set_option('display.max_columns', None)
pd.reset_option('display.max_columns')

pd.set_option('display.max_seq_items', None)

pd.set_option('float_format', '{:.3f}'.format) # 소수점 3개까지 출력

load file

In [3]:
listed = pd.read_csv("../datasets/Train_Test/listed_train_data.csv")
# 영업년수와 상장년수가 있더라

In [4]:
listed.columns

Index(['회사명', '거래소코드', '회계년도', 'CASH FLOW 대 부채비율', 'CASH FLOW 대 차입금비율',
       'CASH FLOW 대 총자본비율', 'CASH FLOW 대 매출액비율', '유동비율', '당좌비율', '부채비율',
       '유동부채비율', '차입금의존도', '순운전자본비율', '현금비율', '유동자산구성비율', '자기자본구성비율',
       '자기자본순이익률', '경영자본순이익률', '매출액순이익률', '총자본사업이익률', '총자본영업이익률', '매출액영업이익률',
       '금융비용부담률', 'CROE', '총자산증가율', '총자본증가율', '유동자산증가율', '비유동자산증가율', '매출액증가율',
       '유형자산증가율', '부가가치', '총자본투자효율', '설비투자효율', '부가가치율', '이윤분배율', '자본분배율',
       '총자산회전율', '매출채권회전률', '당좌자산회전률', '유형자산회전율', '경영자본회전률', '총자본회전률',
       '비유동자산회전률', '유동자산회전률', '영업년수', '상장년수', 'PER', 'PBR', 'PCR', 'PSR',
       '쭈피처', '영업투자현금비율', '영업재무현금비율', '도입기', '성숙기', '성장기', '쇠퇴기', '부실판단'],
      dtype='object')

# describe

all

In [61]:
listed

,회사명,거래소코드,회계년도,CASH FLOW 대 부채비율,CASH FLOW 대 차입금비율,CASH FLOW 대 총자본비율,CASH FLOW 대 매출액비율,유동비율,당좌비율,부채비율,...,쭈피처,영업레버리지,재무레버리지,영업투자현금비율,영업재무현금비율,도입기,성숙기,성장기,쇠퇴기,부실판단
0,성우테크론(주),45300,2014,9,13,4,9,76,76,75,...,0.073,4.102,-0.373,-1.152,15.063,0,0,1,0,1.000
1,(주)동일기연,32960,2022,76,363,5,12,2011,1533,7,...,0.306,9.729,0.939,-0.494,-13.817,0,1,0,0,0.000
2,(주)엔지켐생명과학,183490,2020,-28,-34,-12,-56,1073,979,71,...,0.806,-0.095,0.994,0.310,-0.238,1,0,0,0,1.000
3,유엔젤(주),72130,2022,-2,-8,0,-1,745,730,13,...,0.463,17.670,0.395,-0.736,1.024,0,0,0,1,0.000
4,(주)오이솔루션,138080,2022,-1,-2,0,-1,255,158,27,...,0.414,-2.773,1.349,-1.201,1.217,0,0,0,1,0.000
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
529,한국정밀기계(주),101680,2014,11,21,3,9,383,148,34,...,0.189,0.278,1.044,-1.139,3.854,0,0,1,0,1.000
530,(주)코세스,89890,2015,7,15,6,5,58,40,754,...,0.199,-78.045,0.910,-6.593,-0.222,0,1,0,0,1.000
531,(주)인터로조,119610,2022,18,29,5,10,215,166,39,...,0.205,1.940,1.090,-0.420,1.527,0,0,1,0,0.000
532,(주)엠젠솔루션,32790,2014,4,9,3,1,99,89,242,...,0.201,-15.947,0.130,-0.339,-2.702,0,1,0,0,1.000


In [62]:
listed.describe()

,거래소코드,회계년도,CASH FLOW 대 부채비율,CASH FLOW 대 차입금비율,CASH FLOW 대 총자본비율,CASH FLOW 대 매출액비율,유동비율,당좌비율,부채비율,유동부채비율,...,쭈피처,영업레버리지,재무레버리지,영업투자현금비율,영업재무현금비율,도입기,성숙기,성장기,쇠퇴기,부실판단
count,534.000,534.000,534.000,534.000,534.000,534.000,534.000,534.000,534.000,534.000,...,534.000,534.000,534.000,534.000,531.000,534.000,534.000,534.000,534.000,534.000
mean,138169.607,2019.702,9.307,1664.285,-0.176,-0.968,299.161,247.049,128.135,101.848,...,0.353,0.223,0.631,2.380,13.183,0.298,0.290,0.223,0.189,0.504
std,121484.287,3.015,47.127,26931.452,14.915,29.483,384.260,353.386,419.643,381.869,...,0.486,283.809,2.947,72.371,318.112,0.458,0.454,0.417,0.392,0.500
min,40.000,2014.000,-276.000,-5211.000,-255.000,-451.000,4.000,4.000,0.000,0.000,...,0.015,-5550.949,-47.302,-244.700,-301.491,0.000,0.000,0.000,0.000,0.000
25%,49952.500,2017.000,-10.000,-15.000,-5.000,-8.000,103.000,70.000,32.000,20.000,...,0.127,-0.927,0.421,-1.295,-1.876,0.000,0.000,0.000,0.000,0.000
50%,101205.000,2022.000,2.000,2.000,1.000,1.000,166.000,121.500,72.000,49.000,...,0.219,1.744,0.832,-0.388,-0.467,0.000,0.000,0.000,0.000,1.000
75%,214445.000,2022.000,18.000,35.000,6.000,9.750,319.000,260.500,128.000,100.000,...,0.410,4.151,1.053,0.270,0.221,1.000,1.000,0.000,0.000,1.000
max,900300.000,2022.000,259.000,611057.000,44.000,90.000,3542.000,3246.000,9039.000,8254.000,...,7.116,3404.076,15.017,1529.301,7211.514,1.000,1.000,1.000,1.000,1.000


In [63]:
listed.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 534 entries, 0 to 533
Data columns (total 60 columns):
 #   Column             Non-Null Count  Dtype  
---  ------             --------------  -----  
 0   회사명                534 non-null    object 
 1   거래소코드              534 non-null    int64  
 2   회계년도               534 non-null    int64  
 3   CASH FLOW 대 부채비율   534 non-null    int64  
 4   CASH FLOW 대 차입금비율  534 non-null    int64  
 5   CASH FLOW 대 총자본비율  534 non-null    int64  
 6   CASH FLOW 대 매출액비율  534 non-null    int64  
 7   유동비율               534 non-null    int64  
 8   당좌비율               534 non-null    int64  
 9   부채비율               534 non-null    int64  
 10  유동부채비율             534 non-null    int64  
 11  차입금의존도             534 non-null    int64  
 12  순운전자본비율            534 non-null    int64  
 13  현금비율               534 non-null    int64  
 14  유동자산구성비율           534 non-null    int64  
 15  자기자본구성비율           534 non-null    int64  
 16  자기자본순이익률           534 non

In [64]:
listed.columns

Index(['회사명', '거래소코드', '회계년도', 'CASH FLOW 대 부채비율', 'CASH FLOW 대 차입금비율',
       'CASH FLOW 대 총자본비율', 'CASH FLOW 대 매출액비율', '유동비율', '당좌비율', '부채비율',
       '유동부채비율', '차입금의존도', '순운전자본비율', '현금비율', '유동자산구성비율', '자기자본구성비율',
       '자기자본순이익률', '경영자본순이익률', '매출액순이익률', '총자본사업이익률', '총자본정상영업이익률',
       '매출액정상영업이익률', '금융비용부담률', 'CROE', '총자산증가율', '총자본증가율', '유동자산증가율',
       '비유동자산증가율', '매출액증가율', '유형자산증가율', '부가가치(백만원)', '총자본투자효율', '설비투자효율',
       '부가가치율', '이윤분배율', '자본분배율', '총자산회전율', '매출채권회전률', '당좌자산회전률', '유형자산회전율',
       '경영자본회전률', '총자본회전률', '비유동자산회전률', '유동자산회전률', '영업년수', '상장년수', 'PER',
       'PBR', 'PCR', 'PSR', '쭈피처', '영업레버리지', '재무레버리지', '영업투자현금비율', '영업재무현금비율',
       '도입기', '성숙기', '성장기', '쇠퇴기', '부실판단'],
      dtype='object')

In [65]:
# train data 결측치 개수
listed.isna().sum().sort_values(ascending=False)

영업재무현금비율             3
회사명                  0
영업년수                 0
설비투자효율               0
부가가치율                0
이윤분배율                0
자본분배율                0
총자산회전율               0
매출채권회전률              0
당좌자산회전률              0
유형자산회전율              0
경영자본회전률              0
총자본회전률               0
비유동자산회전률             0
유동자산회전률              0
상장년수                 0
거래소코드                0
PER                  0
PBR                  0
PCR                  0
PSR                  0
쭈피처                  0
영업레버리지               0
재무레버리지               0
영업투자현금비율             0
도입기                  0
성숙기                  0
성장기                  0
쇠퇴기                  0
총자본투자효율              0
부가가치(백만원)            0
유형자산증가율              0
유동자산구성비율             0
회계년도                 0
CASH FLOW 대 부채비율     0
CASH FLOW 대 차입금비율    0
CASH FLOW 대 총자본비율    0
CASH FLOW 대 매출액비율    0
유동비율                 0
당좌비율                 0
부채비율                 0
유동부채비율               0
차입금의존도               0
순운전자본비율    

In [66]:
# train data 0의 개수 비율
listed.drop(['부실판단', '도입기', '성장기', '성숙기', '쇠퇴기'], axis=1).isin([0]).sum().sort_values(ascending=False) / len(listed) * 100

총자본회전률              37.079
경영자본회전률             24.345
이윤분배율               15.356
자본분배율               15.169
비유동자산회전률             9.925
유동자산회전률              8.614
당좌자산회전률              6.554
차입금의존도               6.180
CASH FLOW 대 총자본비율    4.307
비유동자산증가율             2.622
CASH FLOW 대 차입금비율    2.622
총자본증가율               2.434
유형자산회전율              2.434
유형자산증가율              2.434
매출액증가율               2.434
유동자산증가율              2.434
현금비율                 1.873
CASH FLOW 대 매출액비율    1.498
CASH FLOW 대 부채비율     1.124
PBR                  0.936
순운전자본비율              0.936
PSR                  0.936
PCR                  0.936
PER                  0.936
유동부채비율               0.749
부채비율                 0.562
자기자본구성비율             0.562
CROE                 0.375
자기자본순이익률             0.375
총자본정상영업이익률           0.187
총자본투자효율              0.187
부가가치율                0.187
매출채권회전률              0.187
영업레버리지               0.000
상장년수                 0.000
재무레버리지               0.000
쭈피처                  0.000
영

In [67]:
# train data inf 값 개수
listed.isin([np.inf]).sum().sort_values(ascending=False)

회사명                  0
거래소코드                0
설비투자효율               0
부가가치율                0
이윤분배율                0
자본분배율                0
총자산회전율               0
매출채권회전률              0
당좌자산회전률              0
유형자산회전율              0
경영자본회전률              0
총자본회전률               0
비유동자산회전률             0
유동자산회전률              0
영업년수                 0
상장년수                 0
PER                  0
PBR                  0
PCR                  0
PSR                  0
쭈피처                  0
영업레버리지               0
재무레버리지               0
영업투자현금비율             0
영업재무현금비율             0
도입기                  0
성숙기                  0
성장기                  0
쇠퇴기                  0
총자본투자효율              0
부가가치(백만원)            0
유형자산증가율              0
유동자산구성비율             0
회계년도                 0
CASH FLOW 대 부채비율     0
CASH FLOW 대 차입금비율    0
CASH FLOW 대 총자본비율    0
CASH FLOW 대 매출액비율    0
유동비율                 0
당좌비율                 0
부채비율                 0
유동부채비율               0
차입금의존도               0
순운전자본비율    

In [68]:
# train data -inf 값 개수
listed.isin([-np.inf]).sum().sort_values(ascending=False)

회사명                  0
거래소코드                0
설비투자효율               0
부가가치율                0
이윤분배율                0
자본분배율                0
총자산회전율               0
매출채권회전률              0
당좌자산회전률              0
유형자산회전율              0
경영자본회전률              0
총자본회전률               0
비유동자산회전률             0
유동자산회전률              0
영업년수                 0
상장년수                 0
PER                  0
PBR                  0
PCR                  0
PSR                  0
쭈피처                  0
영업레버리지               0
재무레버리지               0
영업투자현금비율             0
영업재무현금비율             0
도입기                  0
성숙기                  0
성장기                  0
쇠퇴기                  0
총자본투자효율              0
부가가치(백만원)            0
유형자산증가율              0
유동자산구성비율             0
회계년도                 0
CASH FLOW 대 부채비율     0
CASH FLOW 대 차입금비율    0
CASH FLOW 대 총자본비율    0
CASH FLOW 대 매출액비율    0
유동비율                 0
당좌비율                 0
부채비율                 0
유동부채비율               0
차입금의존도               0
순운전자본비율    

# 범주 분할 1현금흐름 / 2안정성 / 2수익성 / 3유동성 / 4성장성 / 5생산성 / 6활동성 / 7시장지표 및 기타

In [6]:
# 비상장 all data는 unlisted
# 비상장 label들

labels = listed[
    [
        '거래소코드',
        '회계년도',
        '부실판단',
        '도입기',
        '성숙기',
        '성장기',
        '쇠퇴기'
    ]
]

# 비상장 피처
features = listed.drop(
    labels=[
        '회사명',
        '거래소코드',
        '회계년도',        
        '도입기',
        '성숙기',
        '성장기',
        '쇠퇴기',
        '부실판단'        
    ],
    axis=1,    
)

# 1현금흐름 4개
cash_ft1 = listed[
    [
        'CASH FLOW 대 부채비율',
        'CASH FLOW 대 차입금비율',
        'CASH FLOW 대 총자본비율',
        'CASH FLOW 대 매출액비율',
        '부실판단'
    ]
]

# 2안정성 9개
stability_ft1 = listed[
    [
        '유동비율',
        '당좌비율',
        '부채비율',
        '유동부채비율',
        '차입금의존도',
        '순운전자본비율',
        '현금비율',
        '유동자산구성비율',
        '자기자본구성비율',
        '부실판단'
    ]
]

# 3수익성 8개
profitability_ft1 = listed[
    [
        '자기자본순이익률',
        '경영자본순이익률',
        '매출액순이익률',
        '총자본사업이익률',
        '총자본영업이익률',
        '매출액영업이익률',
        '금융비용부담률',
        'CROE',
        '부실판단'
    ]
]

# 4성장성 6개
growth_ft1 = listed[
    [
        '총자산증가율',
        '총자본증가율',
        '유동자산증가율',
        '비유동자산증가율',
        '매출액증가율',
        '유형자산증가율',
        '부실판단'
    ]
]

# 5생산성 6개
productivity_ft1 = listed[
    [
        '부가가치',
        '총자본투자효율',
        '설비투자효율',
        '부가가치율',
        '이윤분배율',
        '자본분배율',
        '부실판단'
    ]
]

# 6활동성 8개
activity_ft1 = listed[
    [
        '총자산회전율',
        '매출채권회전률',
        '당좌자산회전률',
        '유형자산회전율',
        '경영자본회전률',
        '총자본회전률',
        '비유동자산회전률',
        '유동자산회전률',
        '부실판단'
    ]
]

# 7기타
etc_ft1 = listed[
    [
        '영업년수',
		'상장년수',
		'PBR',
		'PCR',
		'PER',
		'PSR',
        '쭈피처',
        '영업투자현금비율',
        '영업재무현금비율', 
        '도입기',
        '성숙기',
        '성장기',
        '쇠퇴기',
        '부실판단'
    ]
]

In [7]:
listed.columns

Index(['회사명', '거래소코드', '회계년도', 'CASH FLOW 대 부채비율', 'CASH FLOW 대 차입금비율',
       'CASH FLOW 대 총자본비율', 'CASH FLOW 대 매출액비율', '유동비율', '당좌비율', '부채비율',
       '유동부채비율', '차입금의존도', '순운전자본비율', '현금비율', '유동자산구성비율', '자기자본구성비율',
       '자기자본순이익률', '경영자본순이익률', '매출액순이익률', '총자본사업이익률', '총자본영업이익률', '매출액영업이익률',
       '금융비용부담률', 'CROE', '총자산증가율', '총자본증가율', '유동자산증가율', '비유동자산증가율', '매출액증가율',
       '유형자산증가율', '부가가치', '총자본투자효율', '설비투자효율', '부가가치율', '이윤분배율', '자본분배율',
       '총자산회전율', '매출채권회전률', '당좌자산회전률', '유형자산회전율', '경영자본회전률', '총자본회전률',
       '비유동자산회전률', '유동자산회전률', '영업년수', '상장년수', 'PER', 'PBR', 'PCR', 'PSR',
       '쭈피처', '영업투자현금비율', '영업재무현금비율', '도입기', '성숙기', '성장기', '쇠퇴기', '부실판단'],
      dtype='object')

In [8]:
# 1현금흐름 4가지 cash
cash_ft1.describe()

,CASH FLOW 대 부채비율,CASH FLOW 대 차입금비율,CASH FLOW 대 총자본비율,CASH FLOW 대 매출액비율,부실판단
count,533.000000,533.000000,533.000000,533.000000,533.000000
mean,6.667917,1634.499062,-0.236398,-2.358349,0.514071
std,53.941876,26949.701763,15.020926,38.113418,0.500271
min,-606.000000,-5211.000000,-255.000000,-533.000000,0.000000
25%,-10.000000,-15.000000,-4.000000,-8.000000,0.000000
50%,2.000000,3.000000,1.000000,2.000000,1.000000
75%,18.000000,34.000000,6.000000,9.000000,1.000000
max,259.000000,611057.000000,44.000000,90.000000,1.000000


In [9]:
# 2안정성 9가지 stability
stability_ft1.describe()

,유동비율,당좌비율,부채비율,유동부채비율,차입금의존도,순운전자본비율,현금비율,유동자산구성비율,자기자본구성비율,부실판단
count,533.000000,533.000000,533.000000,533.000000,533.000000,533.000000,533.000000,533.000000,533.000000,533.000000
mean,300.842402,249.744841,127.776735,101.048780,24.290807,20.221388,78.525328,55.696060,58.292683,0.514071
std,398.390457,369.296413,421.685029,382.409689,18.118653,28.872805,194.446733,94.886406,21.372473,0.500271
min,4.000000,4.000000,0.000000,0.000000,0.000000,-90.000000,0.000000,3.000000,0.000000,0.000000
25%,103.000000,71.000000,33.000000,21.000000,9.000000,1.000000,14.000000,38.000000,43.000000,0.000000
50%,161.000000,123.000000,71.000000,49.000000,23.000000,19.000000,30.000000,51.000000,58.000000,1.000000
75%,321.000000,259.000000,132.000000,99.000000,37.000000,40.000000,72.000000,65.000000,75.000000,1.000000
max,3542.000000,3246.000000,9039.000000,8254.000000,151.000000,186.000000,2592.000000,2198.000000,96.000000,1.000000


In [10]:
# 3수익성 8가지 profitability
productivity_ft1.describe()

,부가가치,총자본투자효율,설비투자효율,부가가치율,이윤분배율,자본분배율,부실판단
count,533.000000,533.000000,533.000000,533.000000,533.000000,533.000000,533.000000
mean,17413.637899,12.213884,148.515947,19.493433,-409.547842,-234.076923,0.514071
std,27454.697582,24.491122,1595.158149,44.741682,8234.282987,5057.872766,0.500271
min,-56429.000000,-411.000000,-16653.000000,-520.000000,-190075.000000,-116701.000000,0.000000
25%,3808.000000,5.000000,18.000000,10.000000,-32.000000,0.000000,0.000000
50%,10981.000000,13.000000,53.000000,22.000000,0.000000,22.000000,1.000000
75%,24771.000000,22.000000,120.000000,36.000000,27.000000,46.000000,1.000000
max,260543.000000,99.000000,25992.000000,335.000000,292.000000,93.000000,1.000000


In [11]:
# 4성장성 6개 growth
growth_ft1.describe()

,총자산증가율,총자본증가율,유동자산증가율,비유동자산증가율,매출액증가율,유형자산증가율,부실판단
count,533.000000,533.000000,533.000000,533.000000,533.000000,533.000000,533.000000
mean,10.558570,10.177561,9.940075,21.421745,15.938762,108.886417,0.514071
std,41.588706,41.065372,50.018058,108.039620,73.246054,1768.467255,0.500271
min,-86.923616,-80.960000,-97.710000,-95.920000,-95.700000,-99.670000,0.000000
25%,-5.669575,-5.490000,-13.620000,-7.430000,-9.840000,-6.620000,0.000000
50%,3.558418,3.070000,1.580000,2.650000,5.410000,1.080000,1.000000
75%,17.399971,16.130000,19.060000,21.770000,24.030000,19.640000,1.000000
max,443.609526,443.610000,479.050000,1457.130000,1031.410000,40598.090000,1.000000


In [12]:
# 5생산성 6가지 productivity
productivity_ft1.describe()

,부가가치,총자본투자효율,설비투자효율,부가가치율,이윤분배율,자본분배율,부실판단
count,533.000000,533.000000,533.000000,533.000000,533.000000,533.000000,533.000000
mean,17413.637899,12.213884,148.515947,19.493433,-409.547842,-234.076923,0.514071
std,27454.697582,24.491122,1595.158149,44.741682,8234.282987,5057.872766,0.500271
min,-56429.000000,-411.000000,-16653.000000,-520.000000,-190075.000000,-116701.000000,0.000000
25%,3808.000000,5.000000,18.000000,10.000000,-32.000000,0.000000,0.000000
50%,10981.000000,13.000000,53.000000,22.000000,0.000000,22.000000,1.000000
75%,24771.000000,22.000000,120.000000,36.000000,27.000000,46.000000,1.000000
max,260543.000000,99.000000,25992.000000,335.000000,292.000000,93.000000,1.000000


In [13]:
# 6활동성 8가지 activity
activity_ft1.describe()

,총자산회전율,매출채권회전률,당좌자산회전률,유형자산회전율,경영자본회전률,총자본회전률,비유동자산회전률,유동자산회전률,부실판단
count,533.000000,533.000000,533.000000,533.000000,533.000000,533.000000,533.000000,533.000000,533.000000
mean,0.664527,8.155816,2.013133,8.656660,0.844278,0.677298,1.923077,1.418386,0.514071
std,0.422447,7.898677,1.668431,28.517643,0.775658,0.643789,3.081550,0.945607,0.500271
min,0.057989,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
25%,0.382990,4.420000,1.000000,1.000000,1.000000,0.000000,1.000000,1.000000,0.000000
50%,0.584251,6.410000,2.000000,3.000000,1.000000,1.000000,1.000000,1.000000,1.000000
75%,0.876705,9.250000,3.000000,5.000000,1.000000,1.000000,2.000000,2.000000,1.000000
max,3.841965,86.680000,17.000000,408.000000,13.000000,9.000000,52.000000,10.000000,1.000000


In [14]:
# 7기타 etc
etc_ft1.describe()

,영업년수,상장년수,PBR,PCR,PER,PSR,쭈피처,영업투자현금비율,영업재무현금비율,도입기,성숙기,성장기,쇠퇴기,부실판단
count,533.000000,533.000000,5.330000e+02,5.330000e+02,533.000000,5.330000e+02,533.000000,533.000000,531.000000,533.000000,533.000000,533.000000,533.000000,533.000000
mean,27.356473,15.311445,1.165447e+06,-2.509911e+05,-2.921401,5.577589e+05,0.412790,1.794480,-401.874264,0.294559,0.292683,0.212008,0.200750,0.514071
std,13.880686,9.246855,2.279650e+07,4.491568e+06,1075.064344,1.068449e+07,1.280967,67.418733,9518.024741,0.456273,0.455422,0.409114,0.400938,0.500271
min,5.000000,1.000000,-2.269910e+07,-3.935975e+07,-18475.905712,0.000000e+00,0.015304,-207.944436,-219196.000000,0.000000,0.000000,0.000000,0.000000,0.000000
25%,19.000000,8.000000,9.218867e+02,-2.188131e+04,-9.546371,7.844675e+02,0.127763,-1.271075,-1.803401,0.000000,0.000000,0.000000,0.000000,0.000000
50%,25.000000,14.000000,1.498167e+03,2.949515e+03,-0.908265,1.520765e+03,0.219553,-0.397595,-0.447066,0.000000,0.000000,0.000000,0.000000,1.000000
75%,32.000000,22.000000,2.968860e+03,1.537690e+04,17.471910,3.090873e+03,0.416013,0.267836,0.258251,1.000000,1.000000,0.000000,0.000000,1.000000
max,107.000000,51.000000,5.246524e+08,4.238636e+07,8800.000000,2.461992e+08,27.178013,1529.301418,7211.513760,1.000000,1.000000,1.000000,1.000000,1.000000


# 기업수명주기와 부실별

In [15]:
# 도입정상
도입정상 = listed[(listed['도입기'] == 1) & (listed['부실판단'] == 0)]

In [16]:
# 도입정상 결측치확인
도입정상.isna().sum().sort_values(ascending=False)

회사명                  0
유동자산회전률              0
총자본투자효율              0
설비투자효율               0
부가가치율                0
이윤분배율                0
자본분배율                0
총자산회전율               0
매출채권회전률              0
당좌자산회전률              0
유형자산회전율              0
경영자본회전률              0
총자본회전률               0
비유동자산회전률             0
영업년수                 0
거래소코드                0
상장년수                 0
PER                  0
PBR                  0
PCR                  0
PSR                  0
쭈피처                  0
영업투자현금비율             0
영업재무현금비율             0
도입기                  0
성숙기                  0
성장기                  0
쇠퇴기                  0
부가가치                 0
유형자산증가율              0
매출액증가율               0
비유동자산증가율             0
회계년도                 0
CASH FLOW 대 부채비율     0
CASH FLOW 대 차입금비율    0
CASH FLOW 대 총자본비율    0
CASH FLOW 대 매출액비율    0
유동비율                 0
당좌비율                 0
부채비율                 0
유동부채비율               0
차입금의존도               0
순운전자본비율              0
현금비율       

In [17]:
# 도입부실
도입부실 = listed[(listed['도입기'] == 1) & (listed['부실판단'] == 1)]

In [18]:
# 도입부실 결측치확인
도입부실.isna().sum().sort_values(ascending=False)

회사명                  0
유동자산회전률              0
총자본투자효율              0
설비투자효율               0
부가가치율                0
이윤분배율                0
자본분배율                0
총자산회전율               0
매출채권회전률              0
당좌자산회전률              0
유형자산회전율              0
경영자본회전률              0
총자본회전률               0
비유동자산회전률             0
영업년수                 0
거래소코드                0
상장년수                 0
PER                  0
PBR                  0
PCR                  0
PSR                  0
쭈피처                  0
영업투자현금비율             0
영업재무현금비율             0
도입기                  0
성숙기                  0
성장기                  0
쇠퇴기                  0
부가가치                 0
유형자산증가율              0
매출액증가율               0
비유동자산증가율             0
회계년도                 0
CASH FLOW 대 부채비율     0
CASH FLOW 대 차입금비율    0
CASH FLOW 대 총자본비율    0
CASH FLOW 대 매출액비율    0
유동비율                 0
당좌비율                 0
부채비율                 0
유동부채비율               0
차입금의존도               0
순운전자본비율              0
현금비율       

In [19]:
# 성장정상
성장정상 = listed[(listed['성장기'] == 1) & (listed['부실판단'] == 0)]

In [20]:
# 성장정사 결측치확인
성장정상.isna().sum().sort_values(ascending=False)

회사명                  0
유동자산회전률              0
총자본투자효율              0
설비투자효율               0
부가가치율                0
이윤분배율                0
자본분배율                0
총자산회전율               0
매출채권회전률              0
당좌자산회전률              0
유형자산회전율              0
경영자본회전률              0
총자본회전률               0
비유동자산회전률             0
영업년수                 0
거래소코드                0
상장년수                 0
PER                  0
PBR                  0
PCR                  0
PSR                  0
쭈피처                  0
영업투자현금비율             0
영업재무현금비율             0
도입기                  0
성숙기                  0
성장기                  0
쇠퇴기                  0
부가가치                 0
유형자산증가율              0
매출액증가율               0
비유동자산증가율             0
회계년도                 0
CASH FLOW 대 부채비율     0
CASH FLOW 대 차입금비율    0
CASH FLOW 대 총자본비율    0
CASH FLOW 대 매출액비율    0
유동비율                 0
당좌비율                 0
부채비율                 0
유동부채비율               0
차입금의존도               0
순운전자본비율              0
현금비율       

In [21]:
# 성장부실
성장부실 = listed[(listed['성장기'] == 1) & (listed['부실판단'] == 1)]

In [22]:
# 성장부실 결측치 확인
성장부실.isna().sum().sort_values(ascending=False)

회사명                  0
유동자산회전률              0
총자본투자효율              0
설비투자효율               0
부가가치율                0
이윤분배율                0
자본분배율                0
총자산회전율               0
매출채권회전률              0
당좌자산회전률              0
유형자산회전율              0
경영자본회전률              0
총자본회전률               0
비유동자산회전률             0
영업년수                 0
거래소코드                0
상장년수                 0
PER                  0
PBR                  0
PCR                  0
PSR                  0
쭈피처                  0
영업투자현금비율             0
영업재무현금비율             0
도입기                  0
성숙기                  0
성장기                  0
쇠퇴기                  0
부가가치                 0
유형자산증가율              0
매출액증가율               0
비유동자산증가율             0
회계년도                 0
CASH FLOW 대 부채비율     0
CASH FLOW 대 차입금비율    0
CASH FLOW 대 총자본비율    0
CASH FLOW 대 매출액비율    0
유동비율                 0
당좌비율                 0
부채비율                 0
유동부채비율               0
차입금의존도               0
순운전자본비율              0
현금비율       

In [23]:
# 성숙정상
성숙정상 = listed[(listed['성숙기'] == 1) & (listed['부실판단'] == 0)]

In [24]:
# 성숙정상 결측치 확인
성숙정상.isna().sum().sort_values(ascending=False)

회사명                  0
유동자산회전률              0
총자본투자효율              0
설비투자효율               0
부가가치율                0
이윤분배율                0
자본분배율                0
총자산회전율               0
매출채권회전률              0
당좌자산회전률              0
유형자산회전율              0
경영자본회전률              0
총자본회전률               0
비유동자산회전률             0
영업년수                 0
거래소코드                0
상장년수                 0
PER                  0
PBR                  0
PCR                  0
PSR                  0
쭈피처                  0
영업투자현금비율             0
영업재무현금비율             0
도입기                  0
성숙기                  0
성장기                  0
쇠퇴기                  0
부가가치                 0
유형자산증가율              0
매출액증가율               0
비유동자산증가율             0
회계년도                 0
CASH FLOW 대 부채비율     0
CASH FLOW 대 차입금비율    0
CASH FLOW 대 총자본비율    0
CASH FLOW 대 매출액비율    0
유동비율                 0
당좌비율                 0
부채비율                 0
유동부채비율               0
차입금의존도               0
순운전자본비율              0
현금비율       

In [25]:
# 성숙부실
성숙부실 = listed[(listed['성숙기'] == 1) & (listed['부실판단'] == 1)]

In [26]:
# 성숙부실 결측치 확인
성숙부실.isna().sum().sort_values(ascending=False)

회사명                  0
유동자산회전률              0
총자본투자효율              0
설비투자효율               0
부가가치율                0
이윤분배율                0
자본분배율                0
총자산회전율               0
매출채권회전률              0
당좌자산회전률              0
유형자산회전율              0
경영자본회전률              0
총자본회전률               0
비유동자산회전률             0
영업년수                 0
거래소코드                0
상장년수                 0
PER                  0
PBR                  0
PCR                  0
PSR                  0
쭈피처                  0
영업투자현금비율             0
영업재무현금비율             0
도입기                  0
성숙기                  0
성장기                  0
쇠퇴기                  0
부가가치                 0
유형자산증가율              0
매출액증가율               0
비유동자산증가율             0
회계년도                 0
CASH FLOW 대 부채비율     0
CASH FLOW 대 차입금비율    0
CASH FLOW 대 총자본비율    0
CASH FLOW 대 매출액비율    0
유동비율                 0
당좌비율                 0
부채비율                 0
유동부채비율               0
차입금의존도               0
순운전자본비율              0
현금비율       

In [27]:
# 쇠퇴정상
쇠퇴정상 = listed[(listed['쇠퇴기'] == 1) & (listed['부실판단'] == 0)]

In [28]:
# 쇠퇴정상 결측치 확인
쇠퇴정상.isna().sum().sort_values(ascending=False)

회사명                  0
유동자산회전률              0
총자본투자효율              0
설비투자효율               0
부가가치율                0
이윤분배율                0
자본분배율                0
총자산회전율               0
매출채권회전률              0
당좌자산회전률              0
유형자산회전율              0
경영자본회전률              0
총자본회전률               0
비유동자산회전률             0
영업년수                 0
거래소코드                0
상장년수                 0
PER                  0
PBR                  0
PCR                  0
PSR                  0
쭈피처                  0
영업투자현금비율             0
영업재무현금비율             0
도입기                  0
성숙기                  0
성장기                  0
쇠퇴기                  0
부가가치                 0
유형자산증가율              0
매출액증가율               0
비유동자산증가율             0
회계년도                 0
CASH FLOW 대 부채비율     0
CASH FLOW 대 차입금비율    0
CASH FLOW 대 총자본비율    0
CASH FLOW 대 매출액비율    0
유동비율                 0
당좌비율                 0
부채비율                 0
유동부채비율               0
차입금의존도               0
순운전자본비율              0
현금비율       

In [32]:
# 정규성 컬럼
na_col0 = 쇠퇴정상.isna().sum().sort_values(ascending=False).index[0]

# 테스트를 위한 샘플 데이터 생성
data0 = 쇠퇴정상[na_col0].dropna()

In [33]:
# # 샤피로-윌크 검정 수행
# statistic, p_value = shapiro(data0)

# # 검정 통계량과 p-value 출력
# print(f"검정 통계량: {statistic}, p-value: {p_value}")

# # p-value를 기준으로 귀무가설을 기각 여부 판단
# alpha = 0.05
# if p_value > alpha:
#     print("데이터는 정규 분포를 따릅니다 (귀무가설을 기각하지 않음)")
# else:
#     print("데이터는 정규 분포를 따르지 않습니다 (귀무가설을 기각)")

In [34]:
# # 안더슨-달링 검정 수행
# result = anderson(data0)

# # 검정 통계량과 임계값 출력
# print(f"검정 통계량: {result.statistic}")

# # 임계값과 유의수준에 따른 결과 출력
# for i in range(len(result.critical_values)):
#     sl, cv = result.significance_level[i], result.critical_values[i]
#     print(f"유의수준 {sl}%: 임계값 = {cv}")

# # 검정 통계량이 임계값보다 작으면 정규 분포를 따른다고 판단
# if result.statistic < result.critical_values[2]:
#     print("데이터는 정규 분포를 따릅니다.")
# else:
#     print("데이터는 정규 분포를 따르지 않습니다.")

In [36]:
# # Kolmogorov-Smirnov 검정 수행
# statistic, p_value = kstest(data0, 'norm')

# # 검정 통계량과 p-value 출력
# print(f"검정 통계량: {statistic}, p-value: {p_value}")

# # p-value를 기준으로 귀무가설을 기각 여부 판단
# alpha = 0.05
# if p_value > alpha:
#     print("데이터는 정규 분포를 따릅니다 (귀무가설을 기각하지 않음)")
# else:
#     print("데이터는 정규 분포를 따르지 않습니다 (귀무가설을 기각)")

In [38]:
# # fillna median
# 쇠퇴정상[na_col0].fillna(data0.median(), inplace=True)

In [39]:
# 쇠퇴부실
쇠퇴부실 = listed[(listed['쇠퇴기'] == 1) & (listed['부실판단'] == 1)]

In [40]:
# 쇠퇴부실 결측치 확인
쇠퇴부실.isna().sum().sort_values(ascending=False)

영업재무현금비율             2
회사명                  0
유동자산회전률              0
총자본투자효율              0
설비투자효율               0
부가가치율                0
이윤분배율                0
자본분배율                0
총자산회전율               0
매출채권회전률              0
당좌자산회전률              0
유형자산회전율              0
경영자본회전률              0
총자본회전률               0
비유동자산회전률             0
영업년수                 0
거래소코드                0
상장년수                 0
PER                  0
PBR                  0
PCR                  0
PSR                  0
쭈피처                  0
영업투자현금비율             0
도입기                  0
성숙기                  0
성장기                  0
쇠퇴기                  0
부가가치                 0
유형자산증가율              0
매출액증가율               0
비유동자산증가율             0
회계년도                 0
CASH FLOW 대 부채비율     0
CASH FLOW 대 차입금비율    0
CASH FLOW 대 총자본비율    0
CASH FLOW 대 매출액비율    0
유동비율                 0
당좌비율                 0
부채비율                 0
유동부채비율               0
차입금의존도               0
순운전자본비율              0
현금비율       

In [41]:
# 결측치 컬럼
na_col0 = 쇠퇴부실.isna().sum().sort_values(ascending=False).index[0]

# 테스트를 위한 샘플 데이터 생성
data0 = 쇠퇴부실[na_col0].dropna()

In [42]:
# 샤피로-윌크 검정 수행
statistic, p_value = shapiro(data0)

# 검정 통계량과 p-value 출력
print(f"검정 통계량: {statistic}, p-value: {p_value}")

# p-value를 기준으로 귀무가설을 기각 여부 판단
alpha = 0.05
if p_value > alpha:
    print("데이터는 정규 분포를 따릅니다 (귀무가설을 기각하지 않음)")
else:
    print("데이터는 정규 분포를 따르지 않습니다 (귀무가설을 기각)")

검정 통계량: 0.1675984263420105, p-value: 5.001155179998802e-19
데이터는 정규 분포를 따르지 않습니다 (귀무가설을 기각)


In [43]:
# 안더슨-달링 검정 수행
result = anderson(data0)

# 검정 통계량과 임계값 출력
print(f"검정 통계량: {result.statistic}")

# 임계값과 유의수준에 따른 결과 출력
for i in range(len(result.critical_values)):
    sl, cv = result.significance_level[i], result.critical_values[i]
    print(f"유의수준 {sl}%: 임계값 = {cv}")

# 검정 통계량이 임계값보다 작으면 정규 분포를 따른다고 판단
if result.statistic < result.critical_values[2]:
    print("데이터는 정규 분포를 따릅니다.")
else:
    print("데이터는 정규 분포를 따르지 않습니다.")

검정 통계량: 25.43294373869719
유의수준 15.0%: 임계값 = 0.55
유의수준 10.0%: 임계값 = 0.627
유의수준 5.0%: 임계값 = 0.752
유의수준 2.5%: 임계값 = 0.877
유의수준 1.0%: 임계값 = 1.043
데이터는 정규 분포를 따르지 않습니다.


In [44]:
# Kolmogorov-Smirnov 검정 수행
statistic, p_value = kstest(data0, 'norm')

# 검정 통계량과 p-value 출력
print(f"검정 통계량: {statistic}, p-value: {p_value}")

# p-value를 기준으로 귀무가설을 기각 여부 판단
alpha = 0.05
if p_value > alpha:
    print("데이터는 정규 분포를 따릅니다 (귀무가설을 기각하지 않음)")
else:
    print("데이터는 정규 분포를 따르지 않습니다 (귀무가설을 기각)")

검정 통계량: 0.17628945057815126, p-value: 0.012828241793041872
데이터는 정규 분포를 따르지 않습니다 (귀무가설을 기각)


In [45]:
# fillna(median)
쇠퇴부실[na_col0].fillna(data0.median(), inplace=True)

# to_csv

In [46]:
listed.loc[도입부실.index] = 도입부실
listed.loc[도입정상.index] = 도입정상
listed.loc[성장부실.index] = 성장부실
listed.loc[성장정상.index] = 성장정상
listed.loc[성숙부실.index] = 성숙부실
listed.loc[성숙정상.index] = 성숙정상
listed.loc[쇠퇴부실.index] = 쇠퇴부실
listed.loc[쇠퇴정상.index] = 쇠퇴정상

In [47]:
listed.to_csv('../datasets/Train_Test/listed_filled_train_data.csv', index=False)

In [48]:
listed.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 533 entries, 0 to 532
Data columns (total 58 columns):
 #   Column             Non-Null Count  Dtype  
---  ------             --------------  -----  
 0   회사명                533 non-null    object 
 1   거래소코드              533 non-null    int64  
 2   회계년도               533 non-null    int64  
 3   CASH FLOW 대 부채비율   533 non-null    int64  
 4   CASH FLOW 대 차입금비율  533 non-null    int64  
 5   CASH FLOW 대 총자본비율  533 non-null    int64  
 6   CASH FLOW 대 매출액비율  533 non-null    int64  
 7   유동비율               533 non-null    int64  
 8   당좌비율               533 non-null    int64  
 9   부채비율               533 non-null    int64  
 10  유동부채비율             533 non-null    int64  
 11  차입금의존도             533 non-null    int64  
 12  순운전자본비율            533 non-null    int64  
 13  현금비율               533 non-null    int64  
 14  유동자산구성비율           533 non-null    int64  
 15  자기자본구성비율           533 non-null    int64  
 16  자기자본순이익률           533 non

In [49]:
# 결측치 확인
listed.isna().sum().sort_values(ascending=False)

회사명                  0
유동자산회전률              0
총자본투자효율              0
설비투자효율               0
부가가치율                0
이윤분배율                0
자본분배율                0
총자산회전율               0
매출채권회전률              0
당좌자산회전률              0
유형자산회전율              0
경영자본회전률              0
총자본회전률               0
비유동자산회전률             0
영업년수                 0
거래소코드                0
상장년수                 0
PER                  0
PBR                  0
PCR                  0
PSR                  0
쭈피처                  0
영업투자현금비율             0
영업재무현금비율             0
도입기                  0
성숙기                  0
성장기                  0
쇠퇴기                  0
부가가치                 0
유형자산증가율              0
매출액증가율               0
비유동자산증가율             0
회계년도                 0
CASH FLOW 대 부채비율     0
CASH FLOW 대 차입금비율    0
CASH FLOW 대 총자본비율    0
CASH FLOW 대 매출액비율    0
유동비율                 0
당좌비율                 0
부채비율                 0
유동부채비율               0
차입금의존도               0
순운전자본비율              0
현금비율       

In [50]:
# inf 확인
listed.isin([np.inf]).sum().sort_values(ascending=False)

회사명                  0
유동자산회전률              0
총자본투자효율              0
설비투자효율               0
부가가치율                0
이윤분배율                0
자본분배율                0
총자산회전율               0
매출채권회전률              0
당좌자산회전률              0
유형자산회전율              0
경영자본회전률              0
총자본회전률               0
비유동자산회전률             0
영업년수                 0
거래소코드                0
상장년수                 0
PER                  0
PBR                  0
PCR                  0
PSR                  0
쭈피처                  0
영업투자현금비율             0
영업재무현금비율             0
도입기                  0
성숙기                  0
성장기                  0
쇠퇴기                  0
부가가치                 0
유형자산증가율              0
매출액증가율               0
비유동자산증가율             0
회계년도                 0
CASH FLOW 대 부채비율     0
CASH FLOW 대 차입금비율    0
CASH FLOW 대 총자본비율    0
CASH FLOW 대 매출액비율    0
유동비율                 0
당좌비율                 0
부채비율                 0
유동부채비율               0
차입금의존도               0
순운전자본비율              0
현금비율       

In [51]:
# -inf 확인
listed.isin([-np.inf]).sum().sort_values(ascending=False)

회사명                  0
유동자산회전률              0
총자본투자효율              0
설비투자효율               0
부가가치율                0
이윤분배율                0
자본분배율                0
총자산회전율               0
매출채권회전률              0
당좌자산회전률              0
유형자산회전율              0
경영자본회전률              0
총자본회전률               0
비유동자산회전률             0
영업년수                 0
거래소코드                0
상장년수                 0
PER                  0
PBR                  0
PCR                  0
PSR                  0
쭈피처                  0
영업투자현금비율             0
영업재무현금비율             0
도입기                  0
성숙기                  0
성장기                  0
쇠퇴기                  0
부가가치                 0
유형자산증가율              0
매출액증가율               0
비유동자산증가율             0
회계년도                 0
CASH FLOW 대 부채비율     0
CASH FLOW 대 차입금비율    0
CASH FLOW 대 총자본비율    0
CASH FLOW 대 매출액비율    0
유동비율                 0
당좌비율                 0
부채비율                 0
유동부채비율               0
차입금의존도               0
순운전자본비율              0
현금비율       